In [10]:
## TODO: 
## 1. Get mean and std of dataset - done
## 2. Write a script to plot loss + accuracy graph
## 3. FLOPs
## ----------------------------------------------------
## 3. Implement differentiable F1 loss function
## 4. Add class weights
## 5. Implement transfer learning part
## 6. Implement T-SNE
## 7. Implement gradcam
## 8. Ablation study

In [1]:
import sys
sys.path.append('../../src/')

In [2]:
%load_ext autoreload
%autoreload 2

from train import train_model
from model import initialize_model
from utils import set_requires_grad, save_model
from data import load_data
from plotting import plot_data_loader
from eval import eval_model

In [3]:
import os
import copy
import time
import random
import pickle

import numpy as np
import pandas as pd
from PIL import Image
from pathlib import Path
from tqdm import tqdm

from sklearn.metrics import f1_score, confusion_matrix
from numpy.random import shuffle
import matplotlib.pyplot as plt

import torch
import torchvision
from torchvision.io import read_image
import torchvision.transforms as T
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
import torch.autograd.profiler as tprofiler
import torch.utils.data as td

plt.rcParams["savefig.bbox"] = 'tight'

In [4]:
seed = 42

random.seed(seed)

# pytorch RNGs
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

# numpy RNG
np.random.seed(seed)

In [5]:
data_dir = "../../data"
images_dir = "../../data/resized_processed"

In [6]:
# Get best num_workers
# for i in range(97):
#     start = time.time()
#     data_loader = load_data(images_dir,
#                                                                    batch_size = 96, 
#                                                                    input_size = 299, 
#                                                                    norm_arr = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
#                                                                    num_workers = i)
#     iter(data_loader['train']).next()[0].shape
#     print(f"{i}: {time.time()-start}")

In [8]:
# Models options: resnet50, resnet34, inceptionv3, vgg16, mobile_net_v3_large, efficient_net_b1, efficient_net_b0.
model_name = "efficient_net_b0"

# Number of classes.
num_classes = 7

# Batch Size.
batch_size = 64

# Epochs to train for.
num_epochs = 1

# Number of workers for data loader.
num_workers = 8

# Imagenet norm array passed as default value.
norm_arr=([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
# Chest x-ray8 training dataset metrics 
norm_arr=([0.4951, 0.4951, 0.4951], [0.2896, 0.2896, 0.2896])


# Feature extract flag: False - Tune the whole model,
#                       True - Update only the reshaped layer parameters.
feature_extract = False

# Use pretrained flag: None - Use random weights
#                      String - Use pretrained weights given by String
use_pretrained = None

# Initialize the model for this run.
model_pyt, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=use_pretrained)

# lr start and end points for training.
lr_start = 0.01
lr_end = 0.001

# How many epochs to restart.
iter_restart = 10

# Multiplication factor after restart.
mul_restart = 1

# Print the model we just instantiated
#print(model_ft)

In [9]:
data_loaders = load_data(images_dir,
                         batch_size = batch_size, 
                         input_size = input_size, 
                         norm_arr = norm_arr,
                         num_workers = num_workers)

plot_data_loader(data_loader['train'], (2,2))

plot_data_loader(data_loader['test'], (2,2))

plot_data_loader(data_loader['val'], (2,2))

In [10]:
num_layers = 0
for name, param in model_pyt.named_parameters():
    if name.split('.')[-1] == 'bias':
        num_layers += 1
print(f"Number of layers of {model_name}: {num_layers}")

Number of layers of efficient_net_b0: 82


In [12]:
# Check if GPU is available.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# Send model to GPU
model_pyt = model_pyt.to(device)

# Find parameters to be updated in this run.
# parameters with requires_grad = True.
params_to_update = model_pyt.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_pyt.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_pyt.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

cuda:0
Params to learn:
	 features.0.0.weight
	 features.0.1.weight
	 features.0.1.bias
	 features.1.0.block.0.0.weight
	 features.1.0.block.0.1.weight
	 features.1.0.block.0.1.bias
	 features.1.0.block.1.fc1.weight
	 features.1.0.block.1.fc1.bias
	 features.1.0.block.1.fc2.weight
	 features.1.0.block.1.fc2.bias
	 features.1.0.block.2.0.weight
	 features.1.0.block.2.1.weight
	 features.1.0.block.2.1.bias
	 features.2.0.block.0.0.weight
	 features.2.0.block.0.1.weight
	 features.2.0.block.0.1.bias
	 features.2.0.block.1.0.weight
	 features.2.0.block.1.1.weight
	 features.2.0.block.1.1.bias
	 features.2.0.block.2.fc1.weight
	 features.2.0.block.2.fc1.bias
	 features.2.0.block.2.fc2.weight
	 features.2.0.block.2.fc2.bias
	 features.2.0.block.3.0.weight
	 features.2.0.block.3.1.weight
	 features.2.0.block.3.1.bias
	 features.2.1.block.0.0.weight
	 features.2.1.block.0.1.weight
	 features.2.1.block.0.1.bias
	 features.2.1.block.1.0.weight
	 features.2.1.block.1.1.weight
	 features.2.1.block

In [13]:
# conda install -c iopath iopath 
# conda install -c conda-forge yacs  
# conda install -c fvcore -c iopath -c conda-forge fvcore

In [14]:
from fvcore.nn import FlopCountAnalysis

In [15]:
for inputs, labels in data_loaders['train']:
    inputs = inputs.to(device)
    flops = FlopCountAnalysis(model_pyt, inputs)
    print(flops.total())
    break

Unsupported operator aten::add_ encountered 58 time(s)
Unsupported operator aten::silu_ encountered 49 time(s)
Unsupported operator aten::sigmoid encountered 16 time(s)
Unsupported operator aten::mul encountered 25 time(s)
Unsupported operator aten::bernoulli_ encountered 9 time(s)
Unsupported operator aten::div_ encountered 9 time(s)
Unsupported operator aten::dropout_ encountered 1 time(s)
The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
features.1.0.stochastic_depth, features.2.0.stochastic_depth, features.3.0.stochastic_depth, features.4.0.stochastic_depth, features.5.0.stochastic_depth, features.6.0.stochastic_depth, features.7.0.stochastic_depth


8786595840


In [16]:
# 17 min 1 epoch - 128 batch size - inception
# Efficientnet b0 - batch 96 - epoch 50 - num_workers 2 - flip, auto cont, sharp - 

In [ ]:
# Observe that all parameters are being optimized
optimizer = optim.Adam(params_to_update, lr=lr_start)

# Learning rate scheduler.
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=iter_restart, T_mult=mul_restart, 
                                                           eta_min=lr_end, last_epoch=-1)
# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Train and evaluate
model_pyt, prof, val_history, train_history = train_model(device, model_pyt, data_loaders, 
                                                            optimizer, scheduler,
                                                            criterion, 
                                                            num_epochs=num_epochs,
                                                            num_classes=num_classes,
                                                            is_inception=(model_name=="inceptionv3"),
                                                            profiler=False)

In [ ]:
dataset_name = 'chest_xray_8'

In [ ]:
save_model(model_pyt.state_dict(), '../../models/', 
           f'{model_name}_{dataset_name}_{epochs}_{batch_size}_{lr_start}_{lr_end}_model_weights.pth')

In [67]:
eval_metrics = eval_model(device=device, model=model_pyt, test_loader=data_loaders['test'], is_inception=(model_name=="inceptionv3"))

In [76]:
with open(f'../../models/val_history_{model_name}_{dataset_name}_{epochs}_{batch_size}_{lr_start}_{lr_end}_.pickle', 'wb') as handle:
    pickle.dump(val_history, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(f'../../models/train_history_{model_name}_{dataset_name}_{epochs}_{batch_size}_{lr_start}_{lr_end}_.pickle', 'wb') as handle:
    pickle.dump(train_history, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(f'../../models/eval_metrics_{model_name}_{dataset_name}_{epochs}_{batch_size}_{lr_start}_{lr_end}_.pickle', 'wb') as handle:
    pickle.dump(eval_metrics, handle, protocol=pickle.HIGHEST_PROTOCOL)

#with open('filename.pickle', 'rb') as handle:
#    b = pickle.load(handle)

In [ ]:
#print(prof.key_averages(group_by_stack_n=5).table(sort_by='self_cpu_time_total', row_limit=50))

In [9]:
# Plot the training curves of validation accuracy vs. number
#  of training epochs for the transfer learning method and
#  the model trained from scratch
# vhist = []
# vhist = [h.cpu().numpy() for h in val_acc_history]
# thist = []
# thist = [h.cpu().numpy() for h in train_acc_history]

# plt.title("Accuracy vs. Number of Training Epochs")
# plt.xlabel("Training Epochs")
# plt.ylabel("Accuracy")
# #plt.plot(range(1,num_epochs+1),ohist,label="Pretrained")
# plt.plot(range(1,num_epochs+1),vhist,label="Validation")
# plt.plot(range(1,num_epochs+1),thist,label="Training")
# plt.ylim((0,1.))
# plt.xticks(np.arange(1, num_epochs+1, 1.0))
# plt.legend()
# plt.show()

In [8]:
# Plot the training curves of validation accuracy vs. number
#  of training epochs for the transfer learning method and
#  the model trained from scratch
# vhist = []
# vhist = [h for h in val_loss_history]
# thist = []
# thist = [h for h in train_loss_history]

# plt.title("Loss vs. Number of Training Epochs")
# plt.xlabel("Training Epochs")
# plt.ylabel("Loss")
# #plt.plot(range(1,num_epochs+1),ohist,label="Pretrained")
# plt.plot(range(1,num_epochs+1),vhist,label="Validation")
# plt.plot(range(1,num_epochs+1),thist,label="Training")
# plt.ylim((0,1.))
# plt.xticks(np.arange(1, num_epochs+1, 1.0))
# plt.legend()
# plt.show()